<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/example_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/koyomin9zx/UITQA-Vietnamese-Question-Answering.git

Cloning into 'UITQA-Vietnamese-Question-Answering'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 85 (delta 38), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [0]:
!pip install pytorch-transformers
!pip install underthesea 
!pip install unidecode

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ln -s /content/drive/'My Drive'/data

In [0]:
!mv /content/UITQA-Vietnamese-Question-Answering/combine.py /content

In [0]:
# importing libraries 
from underthesea import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial import distance 
from collections import defaultdict, OrderedDict 
from string import punctuation
import unidecode
import re
import pandas as pd 
import numpy as np 
import glob
from combine import QA
import time


In [8]:
start = time.time()
model=QA('/content/data/BERT_Squad_WIki_UIT_pretrain') #path to model
end = time.time()
print("time load model: "+str(round((end - start),2)))

time load model: 35.61


In [0]:
def load_data(path):
  data=[]
  all_files = glob.glob(path + "/*.txt")
  for file in all_files:
      passage=open(file, "r", encoding='utf-8').read()
      data.append(passage)
  return data

def vi_tokenizer(row):
    return word_tokenize(row, format="text")

def remove_stopwords(stopwords,hl_split):
  sent = [s for s in hl_split if s not in stopwords ]
  sent = ' '.join(sent)
  return sent


def remove_punctuation(row):
  remove = punctuation
  remove = remove.replace("_", "")
  pattern = "[{}]".format(remove) # create the pattern
  re_space=re.compile('\s+')
  re_trailing=re.compile('^\s+|\s+?$')
  row=re.sub(pattern, " ", row) 
  row=re.sub(re_space,' ',row)
  row=re.sub(re_trailing,' ',row)
  row = row.strip()
  row =row.lower()
  return row

def standardize_data(df,stopwords):
    hl_cleansed=[]
    remove = punctuation
    remove = remove.replace("_", "")
    pattern = "[{}]".format(remove) # create the pattern
    re_space=re.compile('\s+')
    re_trailing=re.compile('^\s+|\s+?$')
    for row in df:
        #row = vi_tokenizer(row)
        row=re.sub(pattern, " ", row) 
        row=re.sub(re_space,' ',row)
        row=re.sub(re_trailing,' ',row)
        row = row.strip()
        row = remove_stopwords(stopwords,row.split())
        #row = remove_accents(row)
        row = row.lower()
        hl_cleansed.append(row)
    return hl_cleansed


def sentences_tokenize(text):
    sents = sent_tokenize(text)
    sents = [word_tokenize(s,format = 'text') for s in sents]
    sents = [remove_punctuation(s) for s in sents]
    # #sents = [s.lower() for s in sents]
    sents = [remove_stopwords(stopwords,s.split()) for s in sents]
    return sents


## Converting 3D array of array into 1D array 
def arr_convert_1d(arr): 
    arr = np.array(arr) 
    arr = np.concatenate( arr, axis=0 ) 
    arr = np.concatenate( arr, axis=0 ) 
    return arr 
  
## Cosine Similarity 
def cosine(trans): 
    cos = [] 
    cos.append(cosine_similarity(trans[0], trans[1])) 
    return cos

def tfidf(str1, str2,tf_idf_vetor,stopwords):
    str1=standardize_data([str1],stopwords)
    str2=standardize_data([str2],stopwords)  
    corpus = [str1[0],str2[0]] 
    trans = tf_idf_vetor.transform(corpus)
    cos=cosine(trans) 
    return arr_convert_1d(cos)[0]

def relevant_ranking(query,data,vect,stopwords):
  query=standardize_data([query],stopwords)[0]
  print('Query: ',query,'\n')
  score=defaultdict()
  i=0
  for d in data:
    t=tfidf(query, d,vect,stopwords)
    if t!=0.0:
      score[t]=d
    i+=1
  return OrderedDict(sorted(score.items(),reverse=True))



In [124]:
data=load_data('/content/data/data_QA/data')
stopwords = set(open('/content/data/data_QA/stopwords/stopwords.txt').read().split(' ')[:-1])
data_split_sent=[t for i in data for t in sentences_tokenize(i)]
vect = TfidfVectorizer(min_df=1, max_df=0.8,max_features=2000,sublinear_tf=True) 
vect.fit(standardize_data(data,stopwords))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=2000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [142]:
q="Chu Văn An sinh năm nào ?"

a=relevant_ranking(q,data,vect,stopwords)
b=sentences_tokenize(q)[0]
i=0
for score,doc in a.items():
  answer = model.predict(doc,q)
  num_overlap=len(set(b.split()) & set(doc.split()))
  if answer['confidence']>0.7:
    print('\nQuestion: ',q)
    print('\nOverlap key word: ',num_overlap)
    print('\nAnswer: ',answer['answer'])
    print('\nIR Sccore: ',score)
    print('\nBert Score: ',answer['confidence'])
    print('\nContent: ',doc[:100],'...')
    print('\n==========================================\n\n\n')
    i+=1
  if i==3:
    break



Query:  chu văn an sinh năm nào 


Question:  Chu Văn An sinh năm nào ?

Overlap key word:  1

Answer:  1292 - 1370

IR Sccore:  0.11922763416297272

Bert Score:  0.772414214736229

Content:  Chu Văn An (1292 - 1370)

Chu Văn An - Nhà giáo dục đầu tiên của Việt Nam

Chu Văn An (còn gọi là Ch ...





Question:  Chu Văn An sinh năm nào ?

Overlap key word:  1

Answer:  1783

IR Sccore:  0.0761225029846985

Bert Score:  0.9918439475002891

Content:  Nguyễn Quang Toản (sinh Qúi Mão 1783- mất Nhâm Tuất 1802)

Cảnh Thịnh Hoàng đế (Thời gian ở ngôi 179 ...





Question:  Chu Văn An sinh năm nào ?

Overlap key word:  1

Answer:  1809

IR Sccore:  0.07468928148338269

Bert Score:  0.802311580596008

Content:  Cao Bá Quát (Kỉ Tị 1809 – Giáp Dần 1854)

Danh sĩ thời Tự Đức, tự Chu Thần, hiệu Cúc Đường, biệt hiệ ...






In [87]:
def keywords_extraction(sentences):
    sentences=vi_tokenizer(sentences)
    sentences=remove_punctuation(sentences)
    sentences=remove_stopwords(stopwords,sentences.split())
    return sentences.split()

keywords_extraction('năm sinh của chủ tịch nước Việt Nam là gì ?')

['năm', 'sinh', 'chủ_tịch', 'nước', 'việt_nam']